In [9]:
# Import packages
import numpy as np
import scipy as sp
import random as rand
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(suppress=True)

## Benchmark Functions

In [10]:
# Benchmark function, Rosenbrock, global minima at x = 1
def rosen(x):
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

In [11]:
def rosen_with_noise(x, mu, sigma):
    # assume normal distributed noise
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0) + np.random.normal(mu, sigma)

In [12]:
def sphere(x):
    return sum(x**2)

In [13]:
def rastrigin(x):
    return sum(10+x**2-10*np.cos(2*np.pi*x))

## Differential Evolution

In [182]:
def initialize(n, bounds):
    # generates a np array of n rows of random values that lies within the range given by bounds array
    # bounds array is a list of tuples, each tuple indicate the (min value, max_value) for given dimension
    # length of bounds is assumed to define the number of dimensions required
    bounds_range = np.apply_along_axis(lambda x:x[1]-x[0], 1, np.array(bounds))
    bounds_min = np.apply_along_axis(lambda x:x[0], 1, np.array(bounds))
    p_set_norm = np.random.rand(1, len(bounds))
    
    for i in range(n-1):
        p = np.random.rand(1, len(bounds))*bounds_range+bounds_min
        p_set_norm = np.vstack([p_set_norm, p])
        
    p_set = p_set_norm*bounds_range+bounds_min    
    return p_set_norm, p_set

def select_vectors(p_set, r, exclude_i):
    # chooses r vectors from the p_set, excluding the one at index: exclude_i 
    n = len(p_set)    
    idexes = [idx for idx in range(n) if idx != exclude_i]
    return p_set[np.random.choice(idexes, 3, replace=False)]

def get_trial_vector(vectors, c):
    # calculate trial vector using the 3 vector provided
    if len(vectors) != 3:
        return
    return vectors[0] + c*(vectors[1] - vectors[2])

def crossover(parent, trial_vector, pcr):
    # if pcr = 1, means all choose from trial vector, if 0, means all choose from parent vector
    choose_trial = np.random.rand(1,len(trial_vector)) <= pcr
    return (choose_trial*trial_vector + (1-choose_trial)*parent)[0]
    
def differential_evolution(f, bounds, c=0.8, pcr=0.7, n=15, max_iter=1000, 
                           max_eval=1000000, init='latin'):
    # finds minimal for function f subject to boundary given by bounds.    
    # hyperparamters:
    # n random starting positions
    # c is the weight to multiply for the vector differential
    # pcr is the rate of crossover
    
    # initialize n random starting positions
    d = len(bounds) # dimension of inputs
    p_set_norm, p_set = initialize(n, bounds)
    bounds_range = np.apply_along_axis(lambda x:x[1]-x[0], 1, np.array(bounds))
    bounds_min = np.apply_along_axis(lambda x:x[0], 1, np.array(bounds))
    
    # positions variables
    fitness = np.apply_along_axis(f, 1, p_set)
    mean_fitness = np.mean(fitness)
    best_fitness = np.min(fitness)
    best_position = p_set[np.argmin(fitness)]
    best_position_list = []
    
    # loop variables
    terminate = False
    itr = 0
    i_no_improvement = 0
    evals = n
    
    while not terminate:
        for i in range(n):
            # select 3 other vectors from the positions and compute trial vector
            trial_vector = get_trial_vector(select_vectors(p_set_norm, 3, i), c)
            
            # crossover
            offspring_norm = crossover(p_set_norm[i], trial_vector, pcr)
            offspring_norm = np.clip(offspring_norm, 0, 1)
            offspring = offspring_norm*bounds_range+bounds_min 
            
            # select fitter between offspring and parent
            # minimization problem
            offspring_fitness = f(offspring)
            evals += 1
            if offspring_fitness < f(p_set[i]):
                p_set[i] = offspring
                p_set_norm[i] = offspring_norm
                
                if offspring_fitness < best_fitness:
                    best_fitness = offspring_fitness
                    p_set[np.argmin(fitness)] = offspring
                    p_set_norm[np.argmin(fitness)] = offspring_norm
                    fitness[np.argmin(fitness)] = offspring_fitness
                    best_position = offspring
    
        best_position_list.append(best_position)
        
        # check for termination criteria
        if np.mean(fitness) >= mean_fitness:
            i_no_improvement += 1
        else:
            mean_fitness = np.mean(fitness)
            i_no_improvement = 0
        
        if itr > max_iter:
            terminate = True
        elif i_no_improvement > 100:
            terminate = True
        
        if evals > max_eval:
            terminate = True
            
        itr +=1
    
    print('the best solution is {} and function value is {}'.format(best_position, best_fitness))
    print('number of iterations: {}'.format(itr))
    return best_position_list

In [183]:
bounds = [(-10,10)]*4
differential_evolution(rosen, bounds, n=20, max_eval=1000)

the best solution is [0.46891149 0.16576885 0.08459265 0.01804618] and function value is 2.446800327093799
number of iterations: 50


[array([0.46891149, 0.16576885, 0.08459265, 0.01804618]),
 array([-6.        , -4.66035447, -6.        , 10.        ]),
 array([-3.52196443, -5.72828358, -2.8       ,  2.8       ]),
 array([-3.52196443, -5.72828358, -2.8       ,  2.8       ]),
 array([-4.28605723,  5.43586926,  6.3136    , 10.        ]),
 array([1.34594277, 3.81349344, 2.04736   , 2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 1.34594277, -0.17894249,  1.6686848 ,  2.8       ]),
 array([ 0.63523715,  1.00566033, -0.45066765,  2.192     ]),
 array([ 0.63523715,  1.00566033, -0.45066765,  2.192     ]),
 array([ 0.63523

In [185]:
from skopt.space import Space
from skopt.sampler import Lhs

def differential_evolution(f, bounds, mut=0.8, crossp=0.7, popsize=20, 
                           its=1000, init='latin', max_eval=1000000, display=False):
    dimensions = len(bounds)
    
    # initialize positions
    if init == 'random':
        pop = np.random.rand(popsize, dimensions)
    else:
        space = Space([(0.,1.)]*dimensions)
        lhs = Lhs()
        pop = np.asarray(lhs.generate(space.dimensions, popsize))
    
    min_b, max_b = np.asarray(bounds).T
    diff = max_b - min_b
    pop_denorm = min_b + pop * diff
    
    # compute fitness
    fitness = np.apply_along_axis(f, 1, pop_denorm)
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    evals = popsize
    
    # main iteration
    for i in range(its):
        for j in range(popsize):
            # generate 3 other random vectors 
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            
            # compute trial vector and perform boundary check
            # perform cross over
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            trial = cross_points*mutant + (1-cross_points)*pop[j]
            trial_denorm = min_b + trial * diff
            
            # compute fitness of trial vector
            trial_fitness = f(trial_denorm)
            evals += 1
            
            if trial_fitness < fitness[j]:
                fitness[j] = trial_fitness
                pop[j] = trial
                if trial_fitness < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
        if display:
            print('differential evolution step {}: f(x)= {}'.format(i, fitness[best_idx]))
        
        if evals > max_eval:
            break
    print('number of iterations: {}'.format(i))
    return best, fitness[best_idx]

#### Testing on bechmark function

In [186]:
bounds = [(-10,10)]*4
differential_evolution(rosen, bounds, popsize=20, max_eval=1000)

number of iterations: 49


(array([0.75935754, 0.70920394, 0.75827669, 0.53232555]), 8.65875806308)

#### Comparison to Scipy

In [190]:
def callback_DE(xk, convergence):
    if convergence > 1:
        print(xk)
        return True
    else:
        print(xk)
        return False

In [191]:
from scipy.optimize import rosen, differential_evolution
result = differential_evolution(rosen, bounds, popsize=20, maxiter=49, disp=True, callback=callback_DE, polish=False)
result.x, result.fun

differential_evolution step 1: f(x)= 752.698
[ 0.52634998 -1.07038683  3.18561172  8.9408933 ]
differential_evolution step 2: f(x)= 349.571
[-0.10312849  0.10857833  0.59905104  2.12507868]
differential_evolution step 3: f(x)= 349.571
[-0.10312849  0.10857833  0.59905104  2.12507868]
differential_evolution step 4: f(x)= 209.392
[-0.92713544  1.66240494  1.84205818  2.65051208]
differential_evolution step 5: f(x)= 90.2939
[1.1213635  1.47901768 1.26833128 1.53231854]
differential_evolution step 6: f(x)= 42.3204
[ 0.72442501  0.42328948 -0.40204975  0.38822157]
differential_evolution step 7: f(x)= 42.3204
[ 0.72442501  0.42328948 -0.40204975  0.38822157]
differential_evolution step 8: f(x)= 42.3204
[ 0.72442501  0.42328948 -0.40204975  0.38822157]
differential_evolution step 9: f(x)= 42.3204
[ 0.72442501  0.42328948 -0.40204975  0.38822157]
differential_evolution step 10: f(x)= 25.7397
[0.7478027  0.22045544 0.12780284 0.36569043]
differential_evolution step 11: f(x)= 8.04114
[0.569656  

(array([0.99959715, 1.00092858, 0.99551539, 0.98722893]), 0.005805478575822538)

## Genetic Algorithm

In [384]:
def selection(fitness, n):
    # calculate probability of each p being selected as a parent
    f_inv = 1/(fitness/fitness.sum())
    prob = f_inv/f_inv.sum()
    # selecting n set of parents
    indexes = np.argsort(prob)[::-1]
    prob = np.sort(prob)[::-1]
    parents = []
    position = len(fitness)
    for i in range(n):
        parent = []
        for i in range(2):
            r = rand.random()
            for j in range(position):
                if r < prob[j]:
                    parent.append(indexes[j])
                    break
                else:
                    r -= prob[j]
        parents.append(parent)
    return parents
        
def crossover(p_set, parents):
    # keep bits that are similar and perform uniform crossover for the rest
    n = len(parents)
    d = len(p_set[0])
    new_p_set = np.copy(p_set)
    for i in range(n):
        parent_A = p_set[parents[i][0]]
        parent_B = p_set[parents[i][1]]
        
        # go over each dimension and crossover if neccessary
        child_i = []
        for j in range(d):
            A_j = parent_A[j]
            B_j = parent_B[j]
            
            if abs(A_j - B_j) < 0.05:
                child_i.append((A_j + B_j)/2)
            elif rand.random() < 0.5:
                child_i.append(A_j)
            else:
                child_i.append(B_j)
        new_p_set = np.vstack([new_p_set, child_i])
    return new_p_set

def mutation(p_set, pm):
    n = len(p_set)
    d = len(p_set[0])
    for i in range(n):
        for j in range(d):
            if rand.random() < pm:
                p_set[i][j] = (rand.random()*2-1)*2
    return p_set

def survivor(p_set, n, fitness):
    indexes = np.argsort(fitness)[:10]
    fitness = np.sort(fitness)[:10]
    return p_set[indexes]
    
def genetic_optimize(f, d, n, max_iter=1000, max_eval=1000000):
    # genetic algorithm hyperparameters
    # pc: probability of crossover, pm: probability of mutation
    
    pc = 0.4
    pm = 0.6
    
    # initialize and evaluate n potential solutions P(t)
    # limit the search space to +- 10 for each dimension
    
    p_set = (0.5-np.random.rand(1,d))*20
    for i in range(n-1):
        p = (0.5-np.random.rand(1,d))*20
        p_set = np.vstack([p_set, p])
        
    terminate = False
    i = 0
    i_no_improvement = 0
    cur_sol = []
    evals = 0
    
    fitness = np.apply_along_axis(f, 1, p_set)
    pop_fitness = fitness.mean()
    evals += p_set.shape[0]
    
    while not terminate: 
        # Recombine P(t) to form C(t)
        # parent selection
        parents = selection(fitness, round(pc*n))
        # crossover
        p_set = crossover(p_set, parents)
        
        # mutation
        p_set = np.vstack([p_set,mutation(p_set[:n,:],pm)])
        
        # survivor selection
        fitness = np.apply_along_axis(f, 1, p_set)
        p_set = survivor(p_set, n, fitness)
        evals += p_set.shape[0]
        
        # check for termination criteria
        if fitness.mean() >= pop_fitness:
            i_no_improvement += 1
        else:
            i_no_improvement = 0
            pop_fitness = fitness.mean()
                          
        if i > max_iter:
            terminate = True
        elif i_no_improvement > 10:
            terminate = True
        
        i += 1
        
        fitness = np.apply_along_axis(f, 1, p_set)
        cur_sol.append(p_set[np.argsort(fitness)[0]])
        
        if evals > max_eval:
            break
        
    #fitness = np.apply_along_axis(f, 1, p_set)
    #indexes = np.argsort(fitness)
    print('number of iterations: {}'.format(i))
    return cur_sol #p_set[indexes[0]] 

In [385]:
genetic_optimize(rosen, 5, 20, 99)

number of iterations: 57


[array([ 0.60549292,  1.25708193,  0.21237076, -0.9357967 , -0.20077471]),
 array([ 0.60549292,  0.19705181,  0.70557695,  1.17186409, -0.20077471]),
 array([ 0.60549292,  0.19705181,  0.08032394, -0.47901721,  0.0346049 ]),
 array([ 0.60549292,  0.19705181,  0.08032394, -0.47901721,  0.0346049 ]),
 array([ 0.60549292,  0.19705181,  0.08222317, -0.47901721,  0.0346049 ]),
 array([ 0.77625914,  0.19705181,  0.14603683,  0.0202602 , -0.1345386 ]),
 array([ 0.77625914,  0.35458983,  0.14603683,  0.0202602 , -0.1345386 ]),
 array([ 0.77625914,  0.35458983,  0.14603683,  0.0202602 , -0.19955534]),
 array([ 1.00022554,  0.88876749,  0.57700855, -0.19429997,  0.40769615]),
 array([-1.27534381,  0.88876749,  0.57700855,  0.66921407,  0.40769615]),
 array([-0.07480995,  0.94851779,  0.96719353,  0.10286632,  0.02682493]),
 array([-1.03198005,  1.32591427,  0.96719353,  0.33718828, -0.47178732]),
 array([-0.82507128,  1.32591427,  0.96719353,  0.95241192,  0.0445622 ]),
 array([-0.49514149, -0.0

## Simulated Annealing

In [168]:
from scipy.special import gammaln

def get_neighbor_normal(current_state, bounds_range, max_temp, current_temp):
    sd_max = bounds_range/np.sqrt(12)
    sd = sd_max/(0.95 * max_temp) * current_temp + 0.05 * sd_max
    return np.random.normal(current_state, sd)

def get_neighbor_cauchy(current_state, bounds_range, min_bounds, max_bounds, 
                  current_temp, qv):
    factor2 = np.exp((4.0 - qv) * np.log(qv - 1.0))
    factor3 = np.exp((2.0 - qv) * np.log(2.0)/(qv - 1.0))
    factor4_p = np.sqrt(np.pi) * factor2 / factor3 * (3.0 - qv)
    factor5 = 1.0 / (qv - 1.0) - 0.5
    d1 = 2.0 - factor5
    factor6 = np.pi * (1.0 - factor5) / np.sin(np.pi * (1.0 - factor5))/ \
    np.exp(gammaln(d1))
    dim = len(current_state)
    
    x, y = np.random.normal(size=(dim, 2)).T
    factor1 = np.exp(np.log(current_temp) / (qv - 1.0))
    factor4 = factor4_p * factor1
    
    x *= np.exp(-(qv - 1.0) * np.log(factor6 / factor4) / (3.0 - qv))
    den = np.exp((qv - 1.0) * np.log(np.fabs(y)) / (3.0 - qv))

    visits = x / den
    
    TAIL_LIMIT = 1.e8
    MIN_VISIT_BOUND = 1.e-10

    upper_sample, lower_sample = np.random.uniform(size=2)
    visits[visits > TAIL_LIMIT] = TAIL_LIMIT * upper_sample
    visits[visits < TAIL_LIMIT] = TAIL_LIMIT * lower_sample
    x_visit = visits + x
    a = x_visit - min_bounds
    b = np.fmod(a, bounds_range) + bounds_range
    x_visit = np.fmod(b, bounds_range) + min_bounds
    x_visit[np.fabs(x_visit - min_bounds) < MIN_VISIT_BOUND] += 1.e-10
    
    return x_visit
    
def simulated_annealing(f, bounds, initial_temp=5230, temp_func='', 
                        qv=2.62, get_neighbor='cauchy', maxiter=1000, 
                        max_eval=1000000):
    # provide an initial state and temperature
    time = 0
    current_temp = initial_temp
    min_temp = 0.1
    
    current_state = np.random.rand(len(bounds))
    min_b, max_b = np.asarray(bounds).T
    diff = max_b - min_b
    current_state = min_b + current_state * diff

    # evaluate current state
    energy = f(current_state)
    best_energy = energy
    best_state = current_state
    evals = 1
    
    for i in range(maxiter):
        # generate a new state, randomly chosen neighbour of state
        if get_neighbor == 'cauchy':
            neighbor = get_neighbor_cauchy(current_state, diff, min_b, max_b, current_temp, qv)
        else:
            neighbor = get_neighbor_normal(current_state, diff, initial_temp, current_temp)
        
        # evaluate new neighbor
        energy_neighbor = f(neighbor)
        delta = energy_neighbor - energy
        evals += 1
   
        if delta < 0:
            current_state = neighbor
            energy = energy_neighbor
            best_energy = energy
            best_state = current_state
        else:
            if np.random.rand() < np.exp(-delta/current_temp):
                current_state = neighbor
                energy = energy_neighbor
        
        time += 1
        
        alpha = (min_temp/initial_temp)**(1/maxiter)
        
        if temp_func == 'exponential':
            current_temp = initial_temp*np.exp(-time) + 0.000001
        elif temp_func == 'linear':
            current_temp -= (initial_temp-min_temp)/maxiter
        elif temp_func == 'ratio': 
            current_temp = alpha*current_temp
        else:
            current_temp = initial_temp*(2**(qv-1)-1)/((1+time)**(qv-1)-1)
 
    return best_state, best_energy

#### Testing on bechmark function

In [170]:
bounds = [(-10,10)]*50
simulated_annealing(rosen, bounds, max_eval = 1000)

(array([1.02679571, 1.0262024 , 1.02566276, 1.02672742, 1.02612302,
        1.02656624, 1.02639602, 1.02654015, 1.02685423, 1.02611376,
        1.02680802, 1.02631129, 1.02696097, 1.02674777, 1.0268808 ,
        1.02690537, 1.02653987, 1.02647278, 1.02664293, 1.02619786,
        1.02633169, 1.02630398, 1.02638509, 1.02654859, 1.02673646,
        1.02607189, 1.02683491, 1.02634176, 1.02618962, 1.02684825,
        1.02608842, 1.02637982, 1.02641805, 1.0260934 , 1.02667631,
        1.02682146, 1.02634162, 1.02601963, 1.02606674, 1.02631712,
        1.02594888, 1.02675939, 1.02676613, 1.02704107, 1.02686178,
        1.02661508, 1.02661775, 1.02665907, 1.02650972, 1.02679975]),
 3.6599714813122883)

In [192]:
#### Multi-starting point simulated annealing

In [ ]:
from scipy.special import gammaln

def get_neighbor_normal(current_state, bounds_range, max_temp, current_temp):
    sd_max = bounds_range/np.sqrt(12)
    sd = sd_max/(0.95 * max_temp) * current_temp + 0.05 * sd_max
    return np.random.normal(current_state, sd)

def get_neighbor_cauchy(current_state, bounds_range, min_bounds, max_bounds, 
                  current_temp, qv):
    factor2 = np.exp((4.0 - qv) * np.log(qv - 1.0))
    factor3 = np.exp((2.0 - qv) * np.log(2.0)/(qv - 1.0))
    factor4_p = np.sqrt(np.pi) * factor2 / factor3 * (3.0 - qv)
    factor5 = 1.0 / (qv - 1.0) - 0.5
    d1 = 2.0 - factor5
    factor6 = np.pi * (1.0 - factor5) / np.sin(np.pi * (1.0 - factor5))/ \
    np.exp(gammaln(d1))
    dim = len(current_state)
    
    x, y = np.random.normal(size=(dim, 2)).T
    factor1 = np.exp(np.log(current_temp) / (qv - 1.0))
    factor4 = factor4_p * factor1
    
    x *= np.exp(-(qv - 1.0) * np.log(factor6 / factor4) / (3.0 - qv))
    den = np.exp((qv - 1.0) * np.log(np.fabs(y)) / (3.0 - qv))

    visits = x / den
    
    TAIL_LIMIT = 1.e8
    MIN_VISIT_BOUND = 1.e-10

    upper_sample, lower_sample = np.random.uniform(size=2)
    visits[visits > TAIL_LIMIT] = TAIL_LIMIT * upper_sample
    visits[visits < TAIL_LIMIT] = TAIL_LIMIT * lower_sample
    x_visit = visits + x
    a = x_visit - min_bounds
    b = np.fmod(a, bounds_range) + bounds_range
    x_visit = np.fmod(b, bounds_range) + min_bounds
    x_visit[np.fabs(x_visit - min_bounds) < MIN_VISIT_BOUND] += 1.e-10
    
    return x_visit
    
def simulated_annealing(f, bounds, initial_temp=5230, temp_func='', 
                        qv=2.62, get_neighbor='cauchy', maxiter=1000, 
                        max_eval=1000000):
    # provide an initial state and temperature
    time = 0
    current_temp = initial_temp
    min_temp = 0.1
    
    current_state = np.random.rand(len(bounds))
    min_b, max_b = np.asarray(bounds).T
    diff = max_b - min_b
    current_state = min_b + current_state * diff

    # evaluate current state
    energy = f(current_state)
    best_energy = energy
    best_state = current_state
    evals = 1
    
    for i in range(maxiter):
        # generate a new state, randomly chosen neighbour of state
        if get_neighbor == 'cauchy':
            neighbor = get_neighbor_cauchy(current_state, diff, min_b, max_b, current_temp, qv)
        else:
            neighbor = get_neighbor_normal(current_state, diff, initial_temp, current_temp)
        
        # evaluate new neighbor
        energy_neighbor = f(neighbor)
        delta = energy_neighbor - energy
        evals += 1
   
        if delta < 0:
            current_state = neighbor
            energy = energy_neighbor
            best_energy = energy
            best_state = current_state
        else:
            if np.random.rand() < np.exp(-delta/current_temp):
                current_state = neighbor
                energy = energy_neighbor
        
        time += 1
        
        alpha = (min_temp/initial_temp)**(1/maxiter)
        
        if temp_func == 'exponential':
            current_temp = initial_temp*np.exp(-time) + 0.000001
        elif temp_func == 'linear':
            current_temp -= (initial_temp-min_temp)/maxiter
        elif temp_func == 'ratio': 
            current_temp = alpha*current_temp
        else:
            current_temp = initial_temp*(2**(qv-1)-1)/((1+time)**(qv-1)-1)
 
    return best_state, best_energy

## Particle Swarm

In [134]:
from skopt.space import Space
from skopt.sampler import Lhs  

class Particle:
    def __init__(self, bounds, method='latin'):
        self.dimensions = len(bounds)
        self.position = np.empty(self.dimensions)
        self.velocity = np.zeros(self.dimensions)
        self.pbest_position = np.empty(self.dimensions)
        self.pbest_value = None
        self.lowerbounds, self.upperbounds = np.asarray(bounds).T
        
        # initialize positions
        if method == 'random':
            self.position = np.random.rand(self.dimensions)
        elif method == 'latin':
            space = Space([(0.,1.)]*self.dimensions)
            lhs = Lhs()
            self.position = np.asarray(lhs.generate(space.dimensions,1))[0]
    
    def evaluate(self, f):
        cost = f(self.position)

        if self.pbest_value == None:
            self.pbest_value = cost
        elif cost < self.pbest_value:

            if sum(self.upperbounds < self.position)==0 and sum(self.lowerbounds > self.position)==0: 
                self.pbest_value = cost
                self.pbest_position = self.position
    
    def update_velocity(self, gbest, inertia=0.5, c1=1, c2=2):
        # c1: cognitive parameter, c2: social parameter
        self.velocity = inertia*self.velocity + \
                        c1*np.random.rand()*(self.pbest_position-self.position) + \
                        c2*np.random.rand()*(gbest-self.position)
    
    def update_position(self):
        self.position = self.position + self.velocity

In [135]:
p = Particle(bounds)

In [136]:
bounds

[(-10, 10), (-10, 10)]

In [137]:
for i in range(10):
    print(f"cur position {p.position}")
    p.evaluate(sphere)
    print(f"best position {p.pbest_position}, value {p.pbest_value}")
    p.update_velocity(np.array([0.,0.]))
    print(f"velocity {p.velocity}")
    p.update_position()
    print(f"after position {p.position}")

cur position [0.47630672 0.95290308]
best position [1. 1.], value 1.1348923731875253
velocity [-0.56414921 -1.48122477]
after position [-0.08784249 -0.52832169]
cur position [-0.08784249 -0.52832169]
best position [-0.08784249 -0.52832169], value 0.2868401075926256
velocity [-0.17430643 -0.09244922]
after position [-0.26214892 -0.62077091]
cur position [-0.26214892 -0.62077091]
best position [-0.08784249 -0.52832169], value 0.2868401075926256
velocity [0.35028005 0.91343418]
after position [0.08813113 0.29266327]
cur position [0.08813113 0.29266327]
best position [0.08813113 0.29266327], value 0.09341888467046623
velocity [0.16120857 0.41045391]
after position [0.2493397  0.70311717]
cur position [0.2493397  0.70311717]
best position [0.08813113 0.29266327], value 0.09341888467046623
velocity [-0.20920652 -0.57614906]
after position [0.04013318 0.12696812]
cur position [0.04013318 0.12696812]
best position [0.04013318 0.12696812], value 0.01773157520922123
velocity [-0.16473096 -0.4782

In [155]:
def particle_swarm(f, bounds, popsize=20, max_iter=5000, max_eval=1000):
    # initialize particles with random position and velocity
    particles = []
    for i in range(popsize):
        particles.append(Particle(bounds))
    
    gbest_value = None
    gbest_position = np.empty(len(bounds))
    itr = 0
    evals = 0
    
    while itr < max_iter:
        # evaluate cost function and updates personal and global bests
        for i in range(popsize):
            particles[i].evaluate(f)
            evals += 1
            if gbest_value == None or particles[i].pbest_value < gbest_value:
                gbest_value = particles[i].pbest_value
                gbest_position = particles[i].pbest_position
        
        #print(f"gbest value is {gbest_value}")
        # update particles velocity and new positions
        for i in range(popsize):
            particles[i].update_velocity(gbest_position)
            particles[i].update_position()
        
        itr += 1
        
        if evals > max_eval:
            break
        
    return gbest_position, gbest_value
        

In [166]:
bounds = [(-10,10)]*50

In [167]:
particle_swarm(rosen, bounds)

<ipython-input-10-cc7c2886dc2b>:3: RuntimeWarning: overflow encountered in square
  return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)


(array([0.57476937, 0.54510486, 0.54179854, 0.5546148 , 0.46137056,
        0.73300383, 0.65203154, 0.57497845, 0.57078461, 0.52137631,
        0.51255382, 0.41079295, 0.5570628 , 0.3949505 , 0.38671514,
        0.48135654, 0.45502753, 0.40505683, 0.41112971, 0.55655329,
        0.39965213, 0.47343138, 0.49254045, 0.43356959, 0.40991641,
        0.46155804, 0.48451361, 0.46342106, 0.55248351, 0.56168978,
        0.38674346, 0.47726793, 0.58280906, 0.52636326, 0.58854702,
        0.43614154, 0.42208777, 0.58078143, 0.4432474 , 0.50802685,
        0.55306664, 0.46842265, 0.52269295, 0.63106955, 0.43844211,
        0.38621419, 0.53036291, 0.48402731, 0.29401109, 0.42302058]),
 346.3943744922239)

In [ ]:
plt.scatter